### Метод k - близжайших соседей
Мы определяем какой напиток предпочитают на той или иной станции метро: Чай или Кофе.

Для теста вам будет необходимо вести название станции метро без ошибок и с большой буквы(если в названии есть буква "ё", то надо писать через "ё") и надо ввести параметр k который показывает сколько соседних станций учитывабтся при рассчете предпочтений.

In [43]:
import csv
from math import fabs
class Station():
    def __init__(self,names,neighbors,beverage,stationId,lineId):
        self.names=names
        self.neighbors=neighbors
        self.beverage=beverage
        self.coverage_marker=False
        self.stationId=int(stationId)
        self.lineId=int(lineId)
    def remove_repetition(self):
        i=0
        while i<len(self.names):
            j=i+1
            while j <len(self.names):
                if self.names[i]==self.names[j]:
                    del self.names[j]
                else:
                    j+=1
            j=0
            while j<len(self.neighbors):
                if self.names[i]==self.neighbors[j]:
                    del self.neighbors[j]
                else:
                    j+=1
            i+=1
        i=0
        while i<len(self.neighbors):
            j=i+1
            while j<len(self.neighbors):
                if self.neighbors[i]==self.neighbors[j]:
                    del self.neighbors[j]
                else:
                    j+=1
            i+=1
def Stations_PrintList(Stations,line):
    for i in Stations:
        if (line==i.lineId) or (line==0):
            print(i.lineId,i.stationId,i.names,i.neighbors,i.beverage)
def Find_Station(Stations,stationId):
    for station in Stations:
        if station.stationId==stationId:
            return Stations.index(station)
    return -1
def Find_StationName(Stations,name):
    for station in Stations:
        if (name in station.names):
            return Stations.index(station)
    return -1
def Combine_Stations(Stations,transfers_inf):
    for i in range(2,len(transfers_inf)):
        if (transfers_inf[i]!=''):
            station_index1=Find_Station(Stations,int(transfers_inf[1]))
            station_index2=Find_Station(Stations,int(transfers_inf[i]))
            if (station_index1+station_index2>0)and(Stations[station_index1].names[0]!=Stations[station_index2].names[0]):
                Stations[station_index1].names+=Stations[station_index2].names
                Stations[station_index1].neighbors+=Stations[station_index2].neighbors
                del Stations[station_index2]
def Equivalent_Stations(station1,station2):
    for name in station1.names:
        if name in station2.names:
            return True
    return False
def Repetition_Correction(Stations):
    stationId1=0
    while stationId1<len(Stations):
        stationId2=stationId1+1
        while stationId2<len(Stations):
            if Equivalent_Stations(Stations[stationId1],Stations[stationId2]):
                Stations[stationId1].names+=Stations[stationId2].names
                Stations[stationId1].neighbors+=Stations[stationId2].neighbors
                del Stations[stationId2]
            else:
                stationId2+=1
        stationId1+=1
def Add_Beverage(Stations,BeverageFile):
    with open(BeverageFile, "r",encoding="utf8") as f_obj:
        reader = csv.reader(f_obj)
        for bev_inf in reader:
            stationId=Find_StationName(Stations,bev_inf[2])
            if stationId!=-1:
                Stations[stationId].beverage[bev_inf[3].lower()]+=1
def Stations_List(LinesFile,TransfersFile,BeverageFile):
    Stations=[]
    with open(LinesFile, "r",encoding="utf8") as f_obj:
        reader = csv.reader(f_obj)
        for station_inf in reader:
            if station_inf[0]!='' and station_inf[3]!='':
                Stations.append(Station([station_inf[1]],[],{'tea':0,'coffee':0},station_inf[0],station_inf[3]))
    for i in range(0,len(Stations)-1):
        if (Stations[i].lineId==Stations[i+1].lineId) and (fabs(int(Stations[i].stationId)-int(Stations[i+1].stationId))==1):
            Stations[i].neighbors+=Stations[i+1].names
            Stations[i+1].neighbors+=Stations[i].names
    with open(TransfersFile, "r",encoding="utf8") as f_obj:
        reader = csv.reader(f_obj)
        for transfers_inf in reader:
            if transfers_inf[1].isdigit():
                Combine_Stations(Stations,transfers_inf)
    Repetition_Correction(Stations)
    for station in Stations:
        station.remove_repetition()
    Add_Beverage(Stations,BeverageFile)
    return Stations
def Find_Neighbors(Stations,Neighbors):
    Result=[]
    for station in Stations:
        for name in station.names:
            if (name in Neighbors)and(station.coverage_marker==False):
                Result.append(station)
                station.coverage_marker=True
                break
    return Result
def Covered(Stations,station_name,k):
    Station_Covered=[]
    for station in Stations:
        if station_name in station.names:
            Station_Covered.append([station])
            station.coverage_marker=True
    for i in range(0,k):
        Neighbors=[]
        for station in Station_Covered[i]:
            Neighbors+=station.neighbors
        Station_Covered.append(Find_Neighbors(Stations,Neighbors))
    for station in Stations:
        station.coverage_marker=False
    return Station_Covered
def KMethod(Station_Covered):
    Result={'tea':0,'coffee':0}
    for level in Station_Covered:
        for station in level:
            Result['coffee']+=station.beverage['coffee']
            Result['tea']+=station.beverage['tea']
    return Result
def NonLinear_KMethod(Station_Covered):
    Result={'tea':0,'coffee':0}
    for level in Station_Covered:
        if Station_Covered.index(level)==0:
            for station in level:
                Result['coffee']+=(station.beverage['coffee'])
                Result['tea']+=station.beverage['tea']
        else:
            for station in level:
                Result['coffee']+=(station.beverage['coffee'])/Station_Covered.index(level)**2
                Result['tea']+=(station.beverage['tea'])/Station_Covered.index(level)**2
    return Result
def Interface(LinesFile,TransfersFile,BeverageFile,station_name,k):
    Stations=Stations_List(LinesFile,TransfersFile,BeverageFile)
    Station_Covered=Covered(Stations,station_name,k)
    Result=KMethod(Station_Covered)
    if Result['tea']==Result['coffee']:
        Result=NonLinear_KMethod(Station_Covered)
    if Result['tea']>Result['coffee']:
        print('На станции ',station_name,' предпочитают чай')
    elif Result['coffee']>Result['tea']:
        print('На станции ',station_name,' предпочитают кофе')
    else:
        print('На станции ',station_name,' предпочитают в равной степени 2 напитка.')
station = input("введите название станции(вводить с большой буквы и желательно без ошибок):")
k= int(input("А теперь введите параметр k: "))
print()
try :
    Interface('stations_update.csv','transfers.csv','sample.csv',station,k)
except Exception:
    print("Станция введена не правильно")

введите название станции(вводить с большой буквы и желательно без ошибок):Проспект Мира
А теперь введите параметр k: 7

На станции  Проспект Мира  предпочитают чай
